In [ ]:
# import libraries
import pandas as pd
import folium
import json
import geopandas as gpd
import numpy as np
import re

In [6]:
# read each sheet from the excel file (and remove rows with at least 2 NaN values)
filename = 'DSIT 1023 Request ITL2 level split RnD.xlsx'
sheet_1718 = pd.read_excel(filename, sheet_name='2017-18').dropna(axis=0, thresh=2)
sheet_1819 = pd.read_excel(filename, sheet_name='2018-19').dropna(axis=0, thresh=2)
sheet_1920 = pd.read_excel(filename, sheet_name='2019-20').dropna(axis=0, thresh=2)

# cycle through each sheet and clean the data
for sheet, year in zip([sheet_1718, sheet_1819, sheet_1920], ['2017-18', '2018-19', '2019-20']):
    sheet.columns = sheet.iloc[0] # set the header
    sheet.drop(sheet.index[0], inplace=True) # remove the first row
    sheet.rename(columns={'Region': 'ITL221NM'}, inplace=True) # rename the 'Region' column
    sheet.set_index('ITL221NM', inplace=True) # set the index to 'ITL221NM'
    sheet.replace('<5', np.nan, inplace=True) # replace <5 with NaN
    sheet.map(str) # convert all values to strings
    sheet.replace(regex=True, inplace=True, to_replace=r'\D', value=r'') # remove all non-numeric characters
    sheet.apply(pd.to_numeric) # convert all values to numeric
    sheet['Year'] = year # add a column for the year
    sheet.reset_index(inplace=True) # reset the index
    sheet.columns = sheet.columns.str.replace(' ', '_') # replace spaces with underscores
    clean_year = year.replace('-', '_') # replace dashes with underscores
    sheet.to_csv(f'cleaned_{clean_year}.csv', index=False) # save sheet to csv
del sheet, year, clean_year# delete the temporary variables

In [ ]:
# convert coordinates reference system
itl2_gpd = gpd.read_file("International_Territorial_Level_2_January_2021_UK_BUC_V2_2022.geojson") # load geojson file using geodataframe

itl2_gpd.to_crs(epsg=4326, inplace=True) # convert crs to epsg:4326 (latitute and longitude) from epsg:27700 (British National Grid).

itl2_gpd.to_file("itl2_crs_4326.geojson", driver='GeoJSON') # save to geojson file